<a href="https://colab.research.google.com/github/DIFACQUIM/antiviral_ML/blob/main/5_modelability_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Modelability Index (MODI)**
MODI measures the proportion of compounds in a data set whose nearest neighbor belongs to the same class within a defined feature space. We computed the MODI values for ChEMBL data sets using Molecular ACCess System (MACCS) keys (166-bits) and Morgan Chiral of radius 2 (2048-bits) fingerprints, using the RDKit, NumPy, pandas, and SciPy libraries for Python 3. For each target in the ChEMBL data sets, MODI was calculated using two approaches: (1) including compounds classified as “Mixed” in the overall classification, and (2) excluding them.

This section was implemented by Gabriela Valle-Núñez and supervised by Fernanda I. Saldívar-González.

References: https://github.com/rcbraga/modi
[1]: Alexander Golbraikh, Eugene Muratov, Denis Fourches, and Alexander Tropsha Journal of Chemical Information and Modeling 2014 54 (1), 1-4 DOI: 10.1021/ci400572x

**Contact:** [gabrielavallenunez@gmail.com](https://gabrielavallenunez@gmail.com)


# **1. Prepare the environment**

In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 24.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import rdkit as rd
import matplotlib.pyplot as plt
import seaborn as sns
import rdkit.Chem as Chem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.PandasTools import LoadSDF
from rdkit.DataStructs import ConvertToNumpyArray
from scipy import spatial as sp

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [ ]:
import time
tic =time.time()
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        #from platform import python_version
        #if python_version() < "3.7":
        #!pip install https://github.com/biotite-dev/biotite/archive/master.tar.gz

        pbar.update(20)
        #Graphic libraries
        !pip install matplotlib
        import matplotlib.pyplot as plt
        %matplotlib inline
        import matplotlib.font_manager as font_manager
        !pip install seaborn
        import seaborn as sns
        pbar.update(40)

        #System libraries and primary tools
        import os.path
        import os, subprocess
        os.getcwd()
        !pip install cufflinks
        import cufflinks as cf
        import warnings
        warnings.filterwarnings("ignore")
        import math
        from math import pi
        from pathlib import Path
        from tempfile import TemporaryDirectory
        !pip install numpy
        import numpy as np
        !pip install simplejson
        import simplejson as json
        %config Completer.use_jedi = False
        pbar.update(30)
        pbar.update(10)
toc =time.time()
print("Execution time: "+ str(1000*(toc-tic))+" ms")

  0%|          | 0/100 [00:00<?, ?it/s]

Execution time: 39256.696701049805 ms


## **1.1. Load the data set**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/antivirals_machine_learning/Files/DataFrames/7_viralcompound_focusedOnResp_complete_VERSION_2.xlsx - Sheet1 (2).csv", engine='python')
df.head(4)

,Unnamed: 0,molecule_chembl_id,acronym,target,activity_id,assay_chembl_id,assay_description,document_chembl_id,document_journal,ligand_efficiency,...,Overall_Classification,Classification_length,Inhibitor,No_Activity,Unknown,Total,%_Inhibitor,%_No_Activity,%_Unknown,Final_Classification
0,0,CHEMBL222813,IAV,Neuraminidase,3527776 - 6383033 - 3341420 - 13932871 - 13932...,CHEMBL1274484 - CHEMBL1839500 - CHEMBL1167423 ...,Inhibition of Influenza A virus (A/duck/Laos/2...,CHEMBL1134303 - CHEMBL4425127 - CHEMBL3817747 ...,Bioorg Med Chem Lett - Eur J Med Chem - Medche...,"{'bei': '29.48', 'le': '0.58', 'lle': '13.38',...",...,Mixed,2357,179,2,0,181,98.90,1.10,0.00,Inhibitor
1,1,CHEMBL1229,IAV,Neuraminidase,2947997 - 2947996 - 8059690 - 2948009 - 139446...,CHEMBL1041520 - CHEMBL1041523 - CHEMBL1958757 ...,Inhibition of Influenza A virus (A/duck/Laos/2...,CHEMBL4130469 - CHEMBL1134303 - CHEMBL3739372 ...,Bioorg Med Chem Lett - J Nat Prod - Eur J Med ...,"{'bei': '31.36', 'le': '0.61', 'lle': '8.51', ...",...,Mixed,1434,100,6,4,110,90.91,5.45,3.64,Inhibitor
2,2,CHEMBL674,IAV,Neuraminidase,19080701 - 13932882 - 13932879 - 13932877 - 13...,CHEMBL4363628 - CHEMBL3124128 - CHEMBL3124126 ...,Inhibition of Influenza A virus A/Anhui/2005(H...,CHEMBL4364245 - CHEMBL4425127 - CHEMBL3817747 ...,Bioorg Med Chem Lett - Eur J Med Chem - Medche...,"{'bei': '35.02', 'le': '0.68', 'lle': '9.15', ...",...,Mixed,1122,84,2,0,86,97.67,2.33,0.00,Inhibitor
3,3,CHEMBL466246,IAV,Neuraminidase,5086875 - 5086874 - 5086870 - 5086871 - 508777...,CHEMBL1634261 - CHEMBL1634260 - CHEMBL1634256 ...,Inhibition of Influenza A virus (A/Yokohama/67...,CHEMBL1629572 - CHEMBL3232942,Antimicrob Agents Chemother - Bioorg Med Chem,"{'bei': '25.67', 'le': '0.51', 'lle': '11.81',...",...,Inhibitor,572,44,0,0,44,100.00,0.00,0.00,Inhibitor


In [ ]:
# pre-treatment of the data. Unify a column for the target and discard non essential columns
df.columns = df.columns.str.lower()
df = df[['molecule_chembl_id', "acronym", "target", "canonical_smiles_std", "first_pic50", "final_classification"]]
df["unique_target"] = df["acronym"].astype(str) + "_" + df["target"].astype(str).replace(" ", "_")
df.drop("acronym", axis = 1, inplace = True)
df.drop("target", axis = 1, inplace = True)
print(df.columns)
print(df.shape)

Index(['molecule_chembl_id', 'canonical_smiles_std', 'first_pic50',
       'final_classification', 'unique_target'],
      dtype='object')
(4521, 5)


In [ ]:
df = df[(df != 0).all(axis=1)]
print(df)

     molecule_chembl_id                               canonical_smiles_std  \
0          CHEMBL222813  CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...   
1            CHEMBL1229  CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...   
2             CHEMBL674   CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O   
3          CHEMBL466246  CO[C@@H]([C@@H]1OC(C(=O)O)=C[C@H](NC(=N)N)[C@H...   
4          CHEMBL467058  CCCCCCCC(=O)OC[C@@H](O)[C@@H](OC)[C@@H]1OC(C(=...   
...                 ...                                                ...   
4516       CHEMBL495228     COC(=O)c1cc([N+](=O)[O-])c(NC(C)=O)cc1OCCC(C)C   
4517       CHEMBL501838             CC(=O)Nc1cc(OC(C)C)c(C(=O)O)cc1NC(=N)N   
4518      CHEMBL5188555  CN(Cc1cn(C2C3=C(OC2(C)C)c2ccccc2C(=O)C3=O)nn1)...   
4519       CHEMBL495226                   COC(=O)c1ccc(NC(C)=O)cc1OC1CCCC1   
4520       CHEMBL133016  CC(NC(=O)[C@H](CC(=O)N(C)C)NC(=O)[C@@H](NC(=O)...   

      first_pic50 final_classification      unique_target  
0  

In [ ]:
#Rename columns
df.rename(columns = {"canonical_smiles_std":"SMILES", "first_pic50": "pIC50"}, inplace = True)

df.head()

,molecule_chembl_id,SMILES,pIC50,final_classification,unique_target
0,CHEMBL222813,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...,9.795880,Inhibitor,IAV_Neuraminidase
1,CHEMBL1229,CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...,9.795880,Inhibitor,IAV_Neuraminidase
2,CHEMBL674,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O,9.958607,Inhibitor,IAV_Neuraminidase
3,CHEMBL466246,CO[C@@H]([C@@H]1OC(C(=O)O)=C[C@H](NC(=N)N)[C@H...,8.889410,Inhibitor,IAV_Neuraminidase
4,CHEMBL467058,CCCCCCCC(=O)OC[C@@H](O)[C@@H](OC)[C@@H]1OC(C(=...,7.406714,Inhibitor,IAV_Neuraminidase


In [ ]:
# Delete "Mixed" values
df = df[(df != 'Mixed').all(axis=1)]

# Delete "Unknown" values
df = df[(df != 'Unknown').all(axis=1)]

df

,molecule_chembl_id,SMILES,pIC50,final_classification,unique_target
0,CHEMBL222813,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...,9.795880,Inhibitor,IAV_Neuraminidase
1,CHEMBL1229,CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...,9.795880,Inhibitor,IAV_Neuraminidase
2,CHEMBL674,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O,9.958607,Inhibitor,IAV_Neuraminidase
3,CHEMBL466246,CO[C@@H]([C@@H]1OC(C(=O)O)=C[C@H](NC(=N)N)[C@H...,8.889410,Inhibitor,IAV_Neuraminidase
4,CHEMBL467058,CCCCCCCC(=O)OC[C@@H](O)[C@@H](OC)[C@@H]1OC(C(=...,7.406714,Inhibitor,IAV_Neuraminidase
...,...,...,...,...,...
4515,CHEMBL5188623,CC(C)(C)c1ccc(N(C(=O)c2ccco2)C(C(=O)Nc2ccc(F)c...,5.107349,Inhibitor,SARS-CoV-2_Mpro
4516,CHEMBL495228,COC(=O)c1cc([N+](=O)[O-])c(NC(C)=O)cc1OCCC(C)C,5.528708,Inhibitor,IAV_Neuraminidase
4517,CHEMBL501838,CC(=O)Nc1cc(OC(C)C)c(C(=O)O)cc1NC(=N)N,7.309804,Inhibitor,IAV_Neuraminidase
4518,CHEMBL5188555,CN(Cc1cn(C2C3=C(OC2(C)C)c2ccccc2C(=O)C3=O)nn1)...,5.657577,Inhibitor,SARS-CoV-2_PLP


In [ ]:
# Replace "Inhibitor" to "Active" and "No_Activity" to "Inactive"
df['final_classification'] = df['final_classification'].replace({'Inhibitor': 'Active', 'No_Activity': 'Inactive'})
df

,molecule_chembl_id,SMILES,pIC50,final_classification,unique_target
0,CHEMBL222813,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...,9.795880,Active,IAV_Neuraminidase
1,CHEMBL1229,CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...,9.795880,Active,IAV_Neuraminidase
2,CHEMBL674,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O,9.958607,Active,IAV_Neuraminidase
3,CHEMBL466246,CO[C@@H]([C@@H]1OC(C(=O)O)=C[C@H](NC(=N)N)[C@H...,8.889410,Active,IAV_Neuraminidase
4,CHEMBL467058,CCCCCCCC(=O)OC[C@@H](O)[C@@H](OC)[C@@H]1OC(C(=...,7.406714,Active,IAV_Neuraminidase
...,...,...,...,...,...
4515,CHEMBL5188623,CC(C)(C)c1ccc(N(C(=O)c2ccco2)C(C(=O)Nc2ccc(F)c...,5.107349,Active,SARS-CoV-2_Mpro
4516,CHEMBL495228,COC(=O)c1cc([N+](=O)[O-])c(NC(C)=O)cc1OCCC(C)C,5.528708,Active,IAV_Neuraminidase
4517,CHEMBL501838,CC(=O)Nc1cc(OC(C)C)c(C(=O)O)cc1NC(=N)N,7.309804,Active,IAV_Neuraminidase
4518,CHEMBL5188555,CN(Cc1cn(C2C3=C(OC2(C)C)c2ccccc2C(=O)C3=O)nn1)...,5.657577,Active,SARS-CoV-2_PLP


In [ ]:
#Reset index
df = df.reset_index(drop=True)
df

,molecule_chembl_id,SMILES,pIC50,final_classification,unique_target
0,CHEMBL222813,CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...,9.795880,Active,IAV_Neuraminidase
1,CHEMBL1229,CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...,9.795880,Active,IAV_Neuraminidase
2,CHEMBL674,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O,9.958607,Active,IAV_Neuraminidase
3,CHEMBL466246,CO[C@@H]([C@@H]1OC(C(=O)O)=C[C@H](NC(=N)N)[C@H...,8.889410,Active,IAV_Neuraminidase
4,CHEMBL467058,CCCCCCCC(=O)OC[C@@H](O)[C@@H](OC)[C@@H]1OC(C(=...,7.406714,Active,IAV_Neuraminidase
...,...,...,...,...,...
4011,CHEMBL5188623,CC(C)(C)c1ccc(N(C(=O)c2ccco2)C(C(=O)Nc2ccc(F)c...,5.107349,Active,SARS-CoV-2_Mpro
4012,CHEMBL495228,COC(=O)c1cc([N+](=O)[O-])c(NC(C)=O)cc1OCCC(C)C,5.528708,Active,IAV_Neuraminidase
4013,CHEMBL501838,CC(=O)Nc1cc(OC(C)C)c(C(=O)O)cc1NC(=N)N,7.309804,Active,IAV_Neuraminidase
4014,CHEMBL5188555,CN(Cc1cn(C2C3=C(OC2(C)C)c2ccccc2C(=O)C3=O)nn1)...,5.657577,Active,SARS-CoV-2_PLP


# **2. MODI computation**


---



In [ ]:
# Prepare MODI's environment
import os

# Create 'funcs' path
if not os.path.exists('funcs'):
    os.makedirs('funcs')

# Load 'general.py' to 'funcs' path
from google.colab import files
uploaded = files.upload()

# Replace data loaded to 'funcs' path
for filename in uploaded.keys():
    os.rename(filename, os.path.join('funcs', filename))

# Set functions
import sys
import os

# Set path
cwd = os.getcwd() if 'cwd' not in globals() else cwd

try:
    from funcs.general import *  # Import classes and functions needed
except ImportError:
    os.chdir('..')
    cwd = os.getcwd()
    sys.path.insert(0, cwd)
    from funcs.general import *


Saving general.py to general.py
Saving sirms.py to sirms.py


In [ ]:
import sys,os
try:
    if(cwd is not None):
        from funcs.general import *
except:
    %cd ..
    cwd = os.getcwd()
    sys.path.insert(0,cwd)
    from funcs.general import *

## **2.1. MODI computation for each target**


In [ ]:
# List of targets to filter
targets = ['IAV_Neuraminidase', 'SARS-CoV-2_Mpro', 'IBV_Neuraminidase',
           'SARS-CoV-2_PLP', 'HRV_Capsid protein', 'IAV_Polymerase (PA)',
           'IAV_RdRp', 'HRV_Protease', 'SARS-CoV_Mpro',
           'HRSV_Fusion glycoprotein F0', 'SARS-CoV-2_RdRp',
           'IAV_M2 proton channel', 'SARS-CoV-2_Spike glycoprotein',
           'SARS-CoV-2_MTase (NSP14)', 'IAV_Hemagglutinin',
           'IAV_Polymerase (PB2)', 'SARS-CoV-2_Helicase (NSP13)',
           'SARS-CoV_Helicase (NSP13)', 'MERS-CoV_PLP',
           'SARS-CoV_Spike glycoprotein', 'HPIV-1_Hemagglutinin-neuraminidase',
           'NiV_gpG', 'HRV_Mpro', 'HCoV-229E_Mpro', 'HRSV_M2 proton channel',
           'HRSV_Protein P', 'HRSV_RdRp', 'HEV-71_Mpro', 'HEV-71_Capsid protein',
           'FCoV_Mpro', 'MERS-CoV_Mpro', 'HCoV-NL63_PLP']

# Loop through each target and perform the operations
for target in targets:
    # Filter the dataframe for the current target
    target_df = df[df["unique_target"] == target].reset_index(drop=True)
    print(f"Target: {target}")
    print("Shape:", target_df.shape)
    print("Columns:", target_df.columns)
    print(target_df.head(2))

    # Calculate MODI with different descriptors, handle errors for small sample sizes
    try:
        MODI_maccs = modi(target_df, 'final_classification', descriptor="maccs")
        print(f"MODI (maccs) for {target} = {MODI_maccs}")
    except IndexError:
        print(f"Due to data sample, couldn't calculate MODI with maccs for {target}")

    try:
        MODI_morgan = modi(target_df, 'final_classification', descriptor="morgan_chiral2")
        print(f"MODI (morgan_chiral2) for {target} = {MODI_morgan}")
    except IndexError:
        print(f"Due to data sample, couldn't calculate MODI with morgan_chiral2 for {target}")

    # Calculate median pIC50
    median_pIC50 = target_df["pIC50"].median()
    print(f"Median pIC50 for {target} = {median_pIC50}")

    # Group by final_classification and describe pIC50
    pIC50_description = target_df.groupby('final_classification')['pIC50'].describe()
    print(f"pIC50 description for {target}:\n", pIC50_description)

    print("\n" + "="*50 + "\n")  # Separator for readability


Target: IAV_Neuraminidase
Shape: (1123, 5)
Columns: Index(['molecule_chembl_id', 'SMILES', 'pIC50', 'final_classification',
       'unique_target'],
      dtype='object')
  molecule_chembl_id                                             SMILES  \
0       CHEMBL222813  CC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)OC(C(=O)...   
1         CHEMBL1229  CCOC(=O)C1=C[C@@H](OC(CC)CC)[C@H](NC(C)=O)[C@@...   

     pIC50 final_classification      unique_target  
0  9.79588               Active  IAV_Neuraminidase  
1  9.79588               Active  IAV_Neuraminidase  


  0%|          | 0/1123 [00:00<?, ?it/s]

MODI (maccs) for IAV_Neuraminidase = 0.8762243989314337


  0%|          | 0/1123 [00:00<?, ?it/s]

MODI (morgan_chiral2) for IAV_Neuraminidase = 0.9056099732858415
Median pIC50 for IAV_Neuraminidase = 5.721246399
pIC50 description for IAV_Neuraminidase:
                       count      mean       std     min       25%      50%  \
final_classification                                                         
Active                733.0  6.848073  1.260421  5.0000  5.782516  6.60206   
Inactive              390.0  3.775196  1.229119 -3.7404  3.698970  4.02230   

                           75%       max  
final_classification                      
Active                7.853872  9.958607  
Inactive              4.391909  4.950782  


Target: SARS-CoV-2_Mpro
Shape: (815, 5)
Columns: Index(['molecule_chembl_id', 'SMILES', 'pIC50', 'final_classification',
       'unique_target'],
      dtype='object')
  molecule_chembl_id                                             SMILES  \
0      CHEMBL4802135  CC(C)(C)[C@H](NC(=O)C(F)(F)F)C(=O)N1C[C@H]2[C@...   
1      CHEMBL3427166  CC(C)C[C@H](NC(=O

...

MODI (morgan_chiral2) for MERS-CoV_Mpro = 1.0
Median pIC50 for MERS-CoV_Mpro = 6.15490196
pIC50 description for MERS-CoV_Mpro:
                       count      mean       std       min       25%       50%  \
final_classification                                                            
Active                 12.0  6.025958  0.410596  5.124939  6.084122  6.154902   

                           75%      max  
final_classification                     
Active                6.221849  6.39794  


Target: HCoV-NL63_PLP
Shape: (5, 5)
Columns: Index(['molecule_chembl_id', 'SMILES', 'pIC50', 'final_classification',
       'unique_target'],
      dtype='object')
  molecule_chembl_id                                             SMILES  \
0      CHEMBL3233813  C[C@H](c1cccc2ccccc12)N1CCC(C(=O)NCc2ccc(F)c(F...   
1      CHEMBL3233815  C[C@H](c1cccc2ccccc12)N1CCC(C(=O)NCc2cccc(F)c2...   

      pIC50 final_classification  unique_target  
0  4.356547             Inactive  HCoV-NL63_PLP  
1  4.48148

  0%|          | 0/5 [00:00<?, ?it/s]

MODI (maccs) for HCoV-NL63_PLP = 1.0


  0%|          | 0/5 [00:00<?, ?it/s]

MODI (morgan_chiral2) for HCoV-NL63_PLP = 1.0
Median pIC50 for HCoV-NL63_PLP = 4.356547324
pIC50 description for HCoV-NL63_PLP:
                       count      mean       std       min       25%       50%  \
final_classification                                                            
Inactive                5.0  4.367244  0.096571  4.229148  4.337242  4.356547   

                           75%       max  
final_classification                      
Inactive              4.431798  4.481486  




In [ ]:
# List of targets to filter
targets = ['IAV_Neuraminidase', 'SARS-CoV-2_Mpro', 'IBV_Neuraminidase',
           'SARS-CoV-2_PLP', 'HRV_Capsid protein', 'IAV_Polymerase (PA)',
           'IAV_RdRp', 'HRV_Protease', 'SARS-CoV_Mpro',
           'HRSV_Fusion glycoprotein F0', 'SARS-CoV-2_RdRp',
           'IAV_M2 proton channel', 'SARS-CoV-2_Spike glycoprotein',
           'SARS-CoV-2_MTase (NSP14)', 'IAV_Hemagglutinin',
           'IAV_Polymerase (PB2)', 'SARS-CoV-2_Helicase (NSP13)',
           'SARS-CoV_Helicase (NSP13)', 'MERS-CoV_PLP',
           'SARS-CoV_Spike glycoprotein', 'HPIV-1_Hemagglutinin-neuraminidase',
           'NiV_gpG', 'HRV_Mpro', 'HCoV-229E_Mpro', 'HRSV_M2 proton channel',
           'HRSV_Protein P', 'HRSV_RdRp', 'HEV-71_Mpro', 'HEV-71_Capsid protein',
           'FCoV_Mpro', 'MERS-CoV_Mpro', 'HCoV-NL63_PLP']

# Initialize an empty list to store the results
results = []

# Loop through each target and perform the operations
for target in targets:
    # Filter the dataframe for the current target
    target_df = df[df["unique_target"] == target].reset_index(drop=True)

    # Initialize dictionary to store results for the current target
    target_results = {
        "unique_target": target,
        "count": target_df.shape[0],
        "pIC50 median": target_df["pIC50"].median(),
        "Active": target_df[target_df['final_classification'] == 'Active'].shape[0],
        "Inactive": target_df[target_df['final_classification'] == 'Inactive'].shape[0]
    }

    # Calculate MODI with different descriptors, handle errors for small sample sizes
    try:
        target_results["MODI MACCS"] = modi(target_df, 'final_classification', descriptor="maccs")
    except IndexError:
        target_results["MODI MACCS"] = "N/A"

    try:
        target_results["MODI Morgan Chiral2"] = modi(target_df, 'final_classification', descriptor="morgan_chiral2")
    except IndexError:
        target_results["MODI Morgan Chiral2"] = "N/A"

    # Append the results for this target to the list
    results.append(target_results)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Display the resulting DataFrame
print(results_df)


  0%|          | 0/1123 [00:00<?, ?it/s]

  0%|          | 0/1123 [00:00<?, ?it/s]

  0%|          | 0/815 [00:00<?, ?it/s]

  0%|          | 0/815 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

                         unique_target  count  pIC50 median  Active  Inactive  \
0                    IAV_Neuraminidase   1123      5.721246     733       390   
1                      SARS-CoV-2_Mpro    815      6.346787     651       164   
2                    IBV_Neuraminidase    202      5.468709     132        70   
3                       SARS-CoV-2_PLP    107      5.983803      94        13   
4                   HRV_Capsid protein     18      6.265089      15         3   
5                  IAV_Polymerase (PA)    256      5.400000     151       105   
6                             IAV_RdRp    143      3.928118      13       130   
7                         HRV_Protease    389      5.958607     298        91   
8                        SARS-CoV_Mpro    197      4.519993      77       120   
9          HRSV_Fusion glycoprotein F0    249      8.000000     240         9   
10                     SARS-CoV-2_RdRp     46      5.164002      41         5   
11               IAV_M2 prot

In [ ]:
# save results as csv and xlxs
results_df.to_csv("MODI_results_14_11_24.csv")
results_df.to_excel("MODI_results_14_11_24.xlsx")